<a href="https://colab.research.google.com/github/s-miramontes/News_Filter/blob/master/notebooks/summarize_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarize Training Clusters with BERT 

In [0]:
# %%capture
# # Install the latest Tensorflow version.
# !pip3 install --upgrade tensorflow-gpu
# # Install TF-Hub.
# !pip3 install tensorflow-hub

In [0]:
# import statements 

import pandas as pd
import numpy as np

from summarizer import Summarizer

from sklearn.metrics.pairwise import cosine_similarity

from joblib import Parallel, delayed

import heapq
import operator

from absl import logging

import tensorflow as tf
import tensorflow_hub as hub

## Create Summaries for Clusters from Training Data

In [0]:
# import cluster data

# clusters = pd.read_csv("news_filter/data/clusters.csv") 
clusters = pd.read_csv("news_filter/data/filter_clusters.csv") # filtered data

In [0]:
# instantiate summarizer
model = Summarizer()

# function to return summary of each article in cluster
def make_summaries(cluster):
  result = {}
  for i in range(len(cluster.content)):
    summary = model(cluster.content[i], min_length=50, ratio=0.20) 
    result[i] = ''.join(summary)
  return result

In [0]:
# summarize every aritcle in clusters
cluster_summaries = []
for i in range(1,6):
  summaries = make_summaries(clusters[clusters.cluster_labels == i].reset_index())
  cluster_summaries.append(summaries)

In [16]:
cluster_summaries

[{0: 'THE MASSIVE leak of documents from the campaign of Emmanuel Macron failed to prevent his landslide victory Sunday in the French presidential election. What was most striking about the operation was not its impact on the election, but its sheer audacity. Russia must be deterred from waging cyberwar against core democratic institutions of the United States and its NATO allies. An appropriate response must begin with full investigation and disclosure. Marine Le Pen, Mr. Macron’s   opponent, traveled to Moscow to meet Mr. Putin during the campaign and spoke publicly about what Mr. Macron says were fabricated reports that he holds an offshore bank account.',
  1: '(CNN) Sen. Lindsey Graham said Saturday that Russia is trying to ”is trying to break the backs of democracies.” ” Donald Trump’s presidential transition team issued Friday a stunning rebuke of Intelligence communities who have said, in recent reports from CNN and the Washington Post, that Russian hackers not only wanted to d

## Create Summary of Summaries for each Cluster

In [0]:
# summarize summaries of each cluster 
summary_of_summaries = []
for summaries in cluster_summaries:
  summary = ' '.join(list(summaries.values()))
  summary_of_summaries.append(model(summary))

In [18]:
summary_of_summaries

['THE MASSIVE leak of documents from the campaign of Emmanuel Macron failed to prevent his landslide victory Sunday in the French presidential election. An appropriate response must begin with full investigation and disclosure. We call on President Putin to immediately order a halt to this activity. Intelligence Committee members receive classified briefings they can’t speak about in public. WASHINGTON  —   A day after the release of a damning intelligence report on Russia’s   efforts to influence the American election,   Donald J. Trump called on Saturday for a closer relationship between the two nations, saying only “stupid” people or “fools” would think this was unwise. Ukrainian computer specialists mobilized to restore operations in time for the elections. Says electors potentially undermining democracy pic. Toward the beginning of the hearing, Comey said that he has no doubt that Russia attempted to interfere in the 2016 election and that Russian government officials were aware o

## Output Final Summary

In [19]:
input_topics = ["Russian interference with election", "Hillary Clinton WikiLeaks email Benghazi", "Women's march highlights", "Bernie Sander's Campaign", "ICE immigration policies"]

summaries_df = pd.DataFrame({'topic': input_topics, 'summary': summary_of_summaries})
summaries_df

,topic,summary
0,Russian interference with election,THE MASSIVE leak of documents from the campaig...
1,Hillary Clinton WikiLeaks email Benghazi,"To understand the story of Benghazi, you first..."
2,Women's march highlights,That’s what the Internet is agonizing over aft...
3,Bernie Sander's Campaign,Democratic presidential candidate Sen. Bernie ...
4,ICE immigration policies,"For us to continue writing great stories, we n..."


In [0]:
summaries_df.to_csv('news_filter/data/filter_summaries.csv', index=False)